## Mark Klein


## Source 1 -- MySQL

In [44]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [8]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "HaveYouS33nTheBird?"

src_dbname = "sakila"
dst_dbname = "midterm_project"

In [9]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [10]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

#### Extract Data

In [11]:
### Sakila Rental Extraction

sql_rental = "SELECT * FROM sakila.rental;"
df_rental = get_dataframe(user_id, pwd, host_name, src_dbname, sql_rental)
df_rental.head(5)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [12]:
### Sakila Payment Extraction

sql_payment = "SELECT * FROM sakila.payment;"
df_payment = get_dataframe(user_id, pwd, host_name, src_dbname, sql_payment)
df_payment.head(5)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
3,4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
4,5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [13]:
### Sakila Customer Extraction

sql_customer = "SELECT * FROM sakila.customer;"
df_customer = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customer)
df_customer.head(5)

,customer_id,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


#### Transformation

In [14]:
### Rentals Renaming

df_rental.rename(columns={"rental_id":"rental_key"}, inplace=True)
df_rental.rename(columns={"inventory_id":"inventory_key"}, inplace=True)
df_rental.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_rental.rename(columns={"staff_id":"staff_key"}, inplace=True)

df_rental.head(5)

,rental_key,rental_date,inventory_key,customer_key,return_date,staff_key,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [15]:
### Payment Renaming

df_payment.rename(columns={"payment_id":"payment_key"}, inplace=True)
df_payment.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_payment.rename(columns={"staff_id":"staff_key"}, inplace=True)
df_payment.rename(columns={"rental_id":"rental_key"}, inplace=True)

df_payment.head(5)

,payment_key,customer_key,staff_key,rental_key,amount,payment_date,last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30
2,3,1,1,1185,5.99,2005-06-15 00:54:12,2006-02-15 22:12:30
3,4,1,2,1422,0.99,2005-06-15 18:02:53,2006-02-15 22:12:30
4,5,1,2,1476,9.99,2005-06-15 21:08:46,2006-02-15 22:12:30


In [16]:
### Customers Renaming

df_customer.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_customer.rename(columns={"store_id":"store_key"}, inplace=True)
df_customer.rename(columns={"address_id":"address_key"}, inplace=True)

df_customer.head(5)

,customer_key,store_key,first_name,last_name,email,address_key,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20
2,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-14 22:04:36,2006-02-15 04:57:20
3,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-14 22:04:36,2006-02-15 04:57:20
4,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-14 22:04:36,2006-02-15 04:57:20


#### Load Transformation Data

In [17]:
db_operation = "insert"

tables = [('rental', df_rental, 'rental_key'),
          ('customer', df_customer, 'customer_key'),
          ('payment', df_payment, 'payment_key')]

In [18]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### Aggregation

The goal of this aggregation is to determien which customer purchased the most amount of rentals. 

In [19]:
### Aggregate data 

sql_test = """
SELECT SUM(p.`amount`) AS `dollars_of_sales`,
    c.`customer_key` AS `customer_key`,
    c.`last_name` AS `last_name`,
    c.`first_name` AS `first_name`,
    c.`store_key` AS `store_key`
    FROM `{0}`.`payment` AS p
    INNER JOIN `{0}`.`customer` AS c
    ON c.customer_key = p.customer_key
    GROUP BY `customer_key`
    ORDER BY dollars_of_sales DESC;
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test)
df_test.head(5)

,dollars_of_sales,customer_key,last_name,first_name,store_key
0,221.55,526,SEAL,KARL,2
1,216.54,148,HUNT,ELEANOR,1
2,195.58,144,SHAW,CLARA,1
3,194.61,137,KENNEDY,RHONDA,2
4,194.61,178,SNYDER,MARION,2


#### Create Fact Table for Rentals

In [20]:
### Create table

df_rental_facts = pd.merge(df_rental, df_payment, on='rental_key', how='inner')

df_rental_facts.head(5)

,rental_key,rental_date,inventory_key,customer_key_x,return_date,staff_key_x,last_update_x,payment_key,customer_key_y,staff_key_y,amount,payment_date,last_update_y
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,3504,130,1,2.99,2005-05-24 22:53:30,2006-02-15 22:13:16
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,12377,459,2,2.99,2005-05-24 22:54:33,2006-02-15 22:19:16
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,11032,408,2,3.99,2005-05-24 23:03:39,2006-02-15 22:18:00
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,8987,333,1,4.99,2005-05-24 23:04:41,2006-02-15 22:16:17
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,6003,222,1,6.99,2005-05-24 23:05:21,2006-02-15 22:14:21


In [21]:
### Drop unnecessary columns

drop = ['staff_key_y','customer_key_y', 'last_update_x', 'last_update_y']
df_rental_facts.drop(drop, axis=1, inplace=True)

df_rental_facts.head(5)

,rental_key,rental_date,inventory_key,customer_key_x,return_date,staff_key_x,payment_key,amount,payment_date
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,3504,2.99,2005-05-24 22:53:30
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,12377,2.99,2005-05-24 22:54:33
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,11032,3.99,2005-05-24 23:03:39
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,8987,4.99,2005-05-24 23:04:41
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,6003,6.99,2005-05-24 23:05:21


In [22]:
### Rename columns for clarity 

df_rental_facts.rename(columns={"customer_key_x":"customer_key"}, inplace=True)
df_rental_facts.rename(columns={"staff_key_x":"staff_key"}, inplace=True)

df_rental_facts.head(5)

,rental_key,rental_date,inventory_key,customer_key,return_date,staff_key,payment_key,amount,payment_date
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,3504,2.99,2005-05-24 22:53:30
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,12377,2.99,2005-05-24 22:54:33
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,11032,3.99,2005-05-24 23:03:39
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,8987,4.99,2005-05-24 23:04:41
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,6003,6.99,2005-05-24 23:05:21


### Source 2 -- MongoDB

In [23]:
import os
import json
import numpy
import datetime
import pandas as pd 

import pymongo
from sqlalchemy import create_engine

In [24]:
myclient = pymongo.MongoClient("mongodb://localhost/")

midtermdb = myclient["midterm_project"]

inventory = midtermdb["inventory_new"]
customer  = midtermdb["sakila_customer"]
rental = midtermdb["sakila_rental"]


#### Populate with Data

In [25]:
query = {}  # Query to fetch all documents
selection = {'film_id': 1, 'inventory_id': 1, 'last_update': 1}  
cursor = inventory.find(query, selection)
result = list(cursor)
df_inventory = pd.DataFrame(result)
df_inventory.head(2)



,_id,inventory_id,film_id,last_update
0,6540459f8c2c5d8dc4ab0625,1,1,2006-02-15 05:09:17
1,6540459f8c2c5d8dc4ab0626,2,1,2006-02-15 05:09:17


In [26]:
query = {}  
selection = {'active': 1, 'address_id': 1, 'create_data': 1, 'customer_id':1, 'email':1,'first_name':1, 'last_name':1, 
             'last_update':1,'store_id':1}  
cursor = customer.find(query, selection)
result = list(cursor)
df_customer = pd.DataFrame(result)
df_customer.head(2)



,_id,customer_id,store_id,first_name,last_name,email,address_id,active,last_update
0,6540451e8c2c5d8dc4aa8670,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-15 04:57:20
1,6540451e8c2c5d8dc4aa8671,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-15 04:57:20


In [27]:
query = {}  
selection = {'customer_id': 1, 'inventory_id': 1, 'last_update': 1, 'rental_date':1, 'rental_id':1, 
             'return_date':1, 'staff_id':1}  # Including specific fields
cursor = rental.find(query, selection)
result = list(cursor)
df_rental = pd.DataFrame(result)
df_rental.head(2)


,_id,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,654045878c2c5d8dc4aac777,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,654045878c2c5d8dc4aac778,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


#### Transformations

In [28]:
df_inventory.rename(columns={"inventory_id":"inventory_key"}, inplace=True)
df_inventory.rename(columns={"film_id":"film_key"}, inplace=True)
df_inventory.rename(columns={"store_id":"store_key"}, inplace=True)

df_inventory.head(2)

,_id,inventory_key,film_key,last_update
0,6540459f8c2c5d8dc4ab0625,1,1,2006-02-15 05:09:17
1,6540459f8c2c5d8dc4ab0626,2,1,2006-02-15 05:09:17


In [29]:
df_customer.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_customer.rename(columns={"store_id":"store_key"}, inplace=True)
df_customer.rename(columns={"address_id":"address_key"}, inplace=True)

df_customer.head(2)

,_id,customer_key,store_key,first_name,last_name,email,address_key,active,last_update
0,6540451e8c2c5d8dc4aa8670,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-15 04:57:20
1,6540451e8c2c5d8dc4aa8671,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-15 04:57:20


In [30]:
df_rental.rename(columns={"rental_id":"rental_key"}, inplace=True)
df_rental.rename(columns={"inventory_id":"inventory_key"}, inplace=True)
df_rental.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_rental.rename(columns={"staff_id":"staff_key"}, inplace=True)

df_rental.head(2)

,_id,rental_key,rental_date,inventory_key,customer_key,return_date,staff_key,last_update
0,654045878c2c5d8dc4aac777,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,654045878c2c5d8dc4aac778,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


#### Load

In [31]:
dataframe = df_inventory
table_name = 'inventory'
primary_key = 'inventory_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [32]:
dataframe = df_customer
table_name = 'customer'
primary_key = 'customer_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

In [33]:
dataframe = df_rental
table_name = 'rental'
primary_key = 'rental_key'
db_operation = "insert"

set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

#### Validate

In [34]:
import pandas as pd
import pymysql  

def get_sql_dataframe(user_id, pwd, host_name, dst_dbname, query):
    connection = pymysql.connect(host=host_name,
                                 user=user_id,
                                 password=pwd,
                                 database=dst_dbname)
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df

sql_inventory = "SELECT * FROM midterm_project.inventory;"

user_id = 'root'
pwd = 'HaveYouS33nTheBird?'
host_name = 'localhost'
dst_dbname = 'sakila'

df_inventory = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_inventory)
df_inventory.head(5)


/var/folders/hc/xmpxfmqd3pz0s7s2zc_mpxx80000gp/T/ipykernel_72003/997108526.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,_id,inventory_key,film_key,last_update
0,6540459f8c2c5d8dc4ab0625,1,1,2006-02-15 05:09:17
1,6540459f8c2c5d8dc4ab0626,2,1,2006-02-15 05:09:17
2,6540459f8c2c5d8dc4ab0627,3,1,2006-02-15 05:09:17
3,6540459f8c2c5d8dc4ab0628,4,1,2006-02-15 05:09:17
4,6540459f8c2c5d8dc4ab0629,5,1,2006-02-15 05:09:17


In [35]:
def get_sql_dataframe(user_id, pwd, host_name, dst_dbname, query):
    connection = pymysql.connect(host=host_name,
                                 user=user_id,
                                 password=pwd,
                                 database=dst_dbname)
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df

sql_customer = "SELECT * FROM midterm_project.customer;"

user_id = 'root'
pwd = 'HaveYouS33nTheBird?'
host_name = 'localhost'
dst_dbname = 'sakila'

df_customer = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_customer)
df_customer.head(5)


/var/folders/hc/xmpxfmqd3pz0s7s2zc_mpxx80000gp/T/ipykernel_72003/2271154796.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,_id,customer_key,store_key,first_name,last_name,email,address_key,active,last_update
0,6540451e8c2c5d8dc4aa8670,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-15 04:57:20
1,6540451e8c2c5d8dc4aa8671,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-15 04:57:20
2,6540451e8c2c5d8dc4aa8672,3,1,LINDA,WILLIAMS,LINDA.WILLIAMS@sakilacustomer.org,7,1,2006-02-15 04:57:20
3,6540451e8c2c5d8dc4aa8673,4,2,BARBARA,JONES,BARBARA.JONES@sakilacustomer.org,8,1,2006-02-15 04:57:20
4,6540451e8c2c5d8dc4aa8674,5,1,ELIZABETH,BROWN,ELIZABETH.BROWN@sakilacustomer.org,9,1,2006-02-15 04:57:20


In [36]:
def get_sql_dataframe(user_id, pwd, host_name, dst_dbname, query):
    connection = pymysql.connect(host=host_name,
                                 user=user_id,
                                 password=pwd,
                                 database=dst_dbname)
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df

sql_rental = "SELECT * FROM midterm_project.rental;"

user_id = 'root'
pwd = 'HaveYouS33nTheBird?'
host_name = 'localhost'
dst_dbname = 'sakila'

df_rental = get_sql_dataframe(user_id, pwd, host_name, dst_dbname, sql_rental)
df_rental.head(5)


/var/folders/hc/xmpxfmqd3pz0s7s2zc_mpxx80000gp/T/ipykernel_72003/879864651.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,_id,rental_key,rental_date,inventory_key,customer_key,return_date,staff_key,last_update
0,654045878c2c5d8dc4aac777,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,654045878c2c5d8dc4aac778,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,654045878c2c5d8dc4aac779,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,654045878c2c5d8dc4aac77a,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,654045878c2c5d8dc4aac77b,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


#### Aggregation

In [37]:
user_id = 'root'
pwd = 'HaveYouS33nTheBird?'
host_name = 'localhost'
src_dbname = 'sakila'

sql_test2 = """
SELECT r.`return_date` AS `return_date`,
       r.`rental_date` AS `rental_date`,
       r.`rental_key` AS `rental_key`
FROM `midterm_project`.`rental` AS r
GROUP BY r.`rental_key`
ORDER BY return_date DESC;
""".format(dst_dbname)

df_test2 = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test2)
df_test2.head(5)


,return_date,rental_date,rental_key
0,2005-09-02 02:35:22,2005-08-23 21:00:22,16005
1,2005-09-02 02:19:33,2005-08-23 22:19:33,16040
2,2005-09-02 01:28:33,2005-08-23 19:59:33,15971
3,2005-09-01 23:43:24,2005-08-23 18:23:24,15928
4,2005-09-01 22:27:31,2005-08-23 18:07:31,15922


## Source 3 -- Json Read

In [38]:
import json
import numpy as np

with open('/Users/markklein/Desktop/Fall 2023/DS 2002/Data/sakila_inventory.json') as f:
  film = json.load(f)

df_inventory = pd.DataFrame(film)
df_inventory.head(2)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17


In [39]:
with open('/Users/markklein/Desktop/Fall 2023/DS 2002/Data/sakila_store.json') as f:
  store = json.load(f)

df_store = pd.DataFrame(store)
df_store.head()

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [40]:
df_store.rename(columns={"store_id":"store_key"}, inplace=True)
df_store.rename(columns={"manager_staff_id":"manager_staff_key"}, inplace=True)
df_store.rename(columns={"address_id":"address_key"}, inplace=True)

df_store.head()

,store_key,manager_staff_key,address_key,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [41]:
df_inventory.rename(columns={"inventory_id":"inventory_key"}, inplace=True)
df_inventory.rename(columns={"film_id":"film_key"}, inplace=True)
df_inventory.rename(columns={"store_id":"store_key"}, inplace=True)

df_inventory.head(2)

,inventory_key,film_key,store_key,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17


#### Aggregation

In [42]:
sql_test3 = """
SELECT COUNT(*) 
FROM midterm_project.inventory 
WHERE `film_key` = 1;
""".format(dst_dbname)

df_test3 = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test3)

df_test3.head()

,COUNT(*)
0,8


In [43]:
sql_test4 = """
SELECT COUNT(*) 
FROM midterm_project.inventory 
WHERE `film_key` = 2;
""".format(dst_dbname)

df_test4 = get_dataframe(user_id, pwd, host_name, src_dbname, sql_test4)

df_test4.head()

,COUNT(*)
0,3
